In [37]:
import pandas as pd
import numpy as np
import json 
import os
import torch

In [38]:
print(os.getcwd())
style_fake_data = pd.read_json("./gossipcop_v3_keep_data_in_proper_length.json",orient='index')
type(style_fake_data[:20].index)

/mnt/c/Users/wdnmd/PycharmProjects/CSFND_yanjie/gossipcop_glm_origin


pandas.core.indexes.base.Index

In [39]:
style_fake_data['label'].value_counts()

real    11945
fake     3784
Name: label, dtype: int64

In [40]:
def get_all_top_image_id(image_dir_path):
    suffix_to_remove = '_top_img.png'
    all_items  = os.listdir(image_dir_path)
    top_image_ids = {item.rstrip(suffix_to_remove) for item in all_items if os.path.isfile(os.path.join(image_dir_path, item)) and item.endswith(suffix_to_remove)}
    return top_image_ids

def filter_data(data,image_dir_path,use_random=False):
    if use_random:
        return data
    top_image_ids = get_all_top_image_id(image_dir_path)
    print(len(top_image_ids))
    has_top_img_list = [id in top_image_ids for id in data.id]
    return data[has_top_img_list]
    
filter_data = filter_data(style_fake_data,'top_img',True)
# 2.只保留ID  文本内容 标签
keep_features = ["id","text","label"]
keep_data = filter_data[keep_features]



In [41]:
keep_data['label'].value_counts()

real    11945
fake     3784
Name: label, dtype: int64

In [42]:
def split_data(data):
    # 3. 保存到文件中，文本和图像一一对应。
    # 划分训练和测试数据集，要求训练集和测试集中的正负类的分布差不多，
    # 其中 文本 ID 和图像ID一致，也对应上文本的存储位置。
    df_shuffled = data.sample(frac=1.0, random_state=42)
    
    # 计算训练集和测试集的数量
    num_train_samples = 80
    num_val_samples = 80
    num_test_samples = len(df_shuffled) - 160
    
    
    # 分割数据集
    
    train_set = df_shuffled.iloc[:num_train_samples]
    val_set = df_shuffled.iloc[num_train_samples:num_train_samples + num_val_samples]
    test_set = df_shuffled.iloc[num_train_samples + num_val_samples:]
    return train_set, val_set, test_set


data_real_news = keep_data[keep_data.label=='real']
data_fake_news = keep_data[keep_data.label=='fake']
set_len = min(data_real_news.shape[0], data_fake_news.shape[0])
#real_train_data ,real_val_data ,real_test_data = split_data(data_real_news)
#fake_train_data ,fake_val_data ,fake_test_data = split_data(data_fake_news)

real_train_data ,real_val_data ,real_test_data = split_data(data_real_news)
fake_train_data ,fake_val_data ,fake_test_data = split_data(data_fake_news)


train_set = pd.concat([real_train_data,fake_train_data]).sample(frac=1.0, random_state=42)
val_set = pd.concat([real_val_data,fake_val_data]).sample(frac=1.0, random_state=42)
test_set = pd.concat([real_test_data,fake_test_data]).sample(frac=1.0, random_state=42)


train_set['label'].value_counts(),val_set['label'].value_counts(),test_set['label'].value_counts()

(fake    80
 real    80
 Name: label, dtype: int64,
 fake    80
 real    80
 Name: label, dtype: int64,
 real    11785
 fake     3624
 Name: label, dtype: int64)

In [43]:
train_set.to_json("./gossipcop_v3_keep_data_in_proper_length_train.json", orient="records", indent=4,)

In [44]:
val_set.to_json("./gossipcop_v3_keep_data_in_proper_length_valid.json", orient="records", indent=4,)

In [45]:
test_set.to_json("./gossipcop_v3_keep_data_in_proper_length_test.json", orient="records", indent=4,)